# normalise to glucose

In [1]:
# Prerequisites 

from pathlib import Path
import pandas as pd
from ordered_set import OrderedSet
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import yaml
import json
from yaml.loader import SafeLoader
import cobra
from cobra.io import load_model
from cobra import Model, Reaction, Metabolite
from pathlib import Path
from cobra.io import load_json_model, save_json_model, load_matlab_model, save_matlab_model, read_sbml_model, write_sbml_model, load_model, load_yaml_model, save_yaml_model
import logging
from cobra.sampling import sample
from cobra.medium import minimal_medium
import ast

# get carbon number in substrates

In [2]:
glucose_carbons = 6.0
propene_carbons = 3.0
styrene_carbons = 8.0
terephthalate_carbons = 8.0
ethylene_carbons = 2.0
vinyl_chloride_carbons = 2.0

# test glucose growth

In [3]:
# load yeast model
eg_path = Path("/Users/philine/Downloads/yeast-GEM-8.6.2/model")
yml_path = eg_path / "yeast-GEM.yml"
model = load_yaml_model(str(yml_path.resolve()))
model

Restricted license - for non-production use only - expires 2024-10-28


Name,None
Memory address,7fdcacd5ab90
Number of metabolites,2744
Number of reactions,4063
Number of genes,1160
Number of groups,0
Objective expression,1.0*r_2111 - 1.0*r_2111_reverse_58b69
Compartments,"cell envelope, cytoplasm, extracellular, mitochondrion, nucleus, peroxisome, endoplasmic reticulum, Golgi, lipid particle, vacuole, endoplasmic reticulum membrane, vacuolar membrane, Golgi membrane, mitochondrial membrane"


In [4]:
# prep model

def prep_model(model):
    
    model.solver = 'glpk'
    model.objective = 'r_4041'
    
    return model

In [5]:
# identify biomass reaction
model.reactions.get_by_id('r_4041')

Reaction identifier,r_4041
Name,biomass pseudoreaction
Memory address,0x7fdc8f753190
Stoichiometry,55.3 s_0434 + 55.3 s_0803 + s_1096 + s_3717 + s_3718 + s_3719 + s_3720 + s_4205 + s_4206 --> 55.3 s_0394 + s_0450 + 55.3 s_0794 + 55.3 s_1322 55.3 ATP + 55.3 H2O + lipid + protein + carbohydrate + RNA + DNA + cofactor + ion --> 55.3 ADP + biomass + 55.3 H+ + 55.3 phosphate
GPR,
Lower bound,0.0
Upper bound,1000.0


In [6]:
# apply function to glucose model
glucose = prep_model(model)

In [7]:
glucose_solution = model.optimize()
glucose_solution

,fluxes,reduced_costs
r_0001,0.000000,-1.384956e-17
r_0002,0.000000,-6.938894e-18
r_0003,0.000000,-1.549268e-17
r_0004,0.000000,0.000000e+00
r_0005,0.061324,1.387779e-17
...,...,...
r_4706,0.000000,-0.000000e+00
r_4707,0.000000,0.000000e+00
r_4708,0.000000,0.000000e+00
r_4709,0.000000,0.000000e+00


In [8]:
# get objective value of glucose
glucose_growth_temp = glucose_solution.objective_value
glucose_growth_temp

0.08192806489430318

In [9]:
# rounded objective value
glucose_growth = glucose_growth_temp
glucose_growth = round(glucose_growth, 6)
glucose_growth

0.081928

In [10]:
# normalise growth by carbon
glucose_normalise = glucose_growth/glucose_carbons
glucose_normalise = round(glucose_normalise, 6)

glucose_normalise

0.013655

In [11]:
# read df of fba results
growth_table = pd.read_csv('table.csv')
growth_table['solution'] = growth_table['solution'].round(6)
growth_table

,source,target,solution,rounded solution,length
0,Vinyl chloride,Naringenin,9.379374,9.379374,6
1,Vinyl chloride,Naringenin,0.000000,0.000000,6
2,Vinyl chloride,Naringenin,9.379374,9.379374,6
3,Vinyl chloride,Apigenin,9.379374,9.379374,6
4,Vinyl chloride,Apigenin,9.379374,9.379374,6
5,Vinyl chloride,Pinocembrin,0.000000,0.000000,6
6,Vinyl chloride,beta-Carotene,6.126559,6.126559,6
7,Vinyl chloride,beta-Carotene,6.126559,6.126559,6
8,Vinyl chloride,beta-Carotene,6.126559,6.126559,6
9,Vinyl chloride,beta-Carotene,6.322964,6.322964,6


In [12]:
# group together by source, target and solution
grouped_growth_table = growth_table.groupby(['source', 'target', 'solution']).size().reset_index(name='number of pathways')
grouped_growth_table

,source,target,solution,number of pathways
0,Propene,Succinate,2.586895,1
1,Propene,beta-Carotene,1.795530,2
2,Vinyl chloride,Apigenin,9.379374,2
3,Vinyl chloride,Eriodictyol,0.000000,1
4,Vinyl chloride,Luteolin,9.379374,2
5,Vinyl chloride,Naringenin,0.000000,1
6,Vinyl chloride,Naringenin,9.379374,2
7,Vinyl chloride,Pinocembrin,0.000000,1
8,Vinyl chloride,beta-Carotene,1.795530,6
9,Vinyl chloride,beta-Carotene,6.126559,16


In [13]:
# order by size of solution value
grouped_growth_table = grouped_growth_table.sort_values(by='solution', ascending=False)
grouped_growth_table

,source,target,solution,number of pathways
2,Vinyl chloride,Apigenin,9.379374,2
4,Vinyl chloride,Luteolin,9.379374,2
6,Vinyl chloride,Naringenin,9.379374,2
10,Vinyl chloride,beta-Carotene,6.322964,8
9,Vinyl chloride,beta-Carotene,6.126559,16
0,Propene,Succinate,2.586895,1
1,Propene,beta-Carotene,1.795530,2
8,Vinyl chloride,beta-Carotene,1.795530,6
3,Vinyl chloride,Eriodictyol,0.000000,1
5,Vinyl chloride,Naringenin,0.000000,1


In [14]:
# normalise all values and add to df
grouped_growth_table['normalised values'] = 0.0

for i, row in grouped_growth_table.iterrows():
    if row['source'] == 'Propene':
        grouped_growth_table.at[i, 'normalised values'] = row['solution'] / float(propene_carbons)
        
    elif row['source'] == 'Vinyl chloride':
        grouped_growth_table.at[i, 'normalised values'] = row['solution'] / float(vinyl_chloride_carbons)
        
    elif row['source'] == 'Ethylene':
        grouped_growth_table.at[i, 'normalised values'] = row['solution'] / float(ethylene_carbons)
        
    elif row['source'] == 'Styrene':
        grouped_growth_table.at[i, 'normalised values'] = row['solution'] / float(styrene_carbons)
        
    else:
        grouped_growth_table.at[i, 'normalised values'] = row['solution'] / float(terephthalate_carbons)
        
grouped_growth_table

,source,target,solution,number of pathways,normalised values
2,Vinyl chloride,Apigenin,9.379374,2,4.689687
4,Vinyl chloride,Luteolin,9.379374,2,4.689687
6,Vinyl chloride,Naringenin,9.379374,2,4.689687
10,Vinyl chloride,beta-Carotene,6.322964,8,3.161482
9,Vinyl chloride,beta-Carotene,6.126559,16,3.063280
0,Propene,Succinate,2.586895,1,0.862298
1,Propene,beta-Carotene,1.795530,2,0.598510
8,Vinyl chloride,beta-Carotene,1.795530,6,0.897765
3,Vinyl chloride,Eriodictyol,0.000000,1,0.000000
5,Vinyl chloride,Naringenin,0.000000,1,0.000000


In [15]:
# add columns stating if growth higher than growth with glucose
grouped_growth_table['higher yield than glucose'] = 0

for i, row in grouped_growth_table.iterrows():
    if row['normalised values'] < glucose_normalise:
        grouped_growth_table.at[i, 'higher yield than glucose'] = 0
    else:
        grouped_growth_table.at[i, 'higher yield than glucose'] = 1
        
grouped_growth_table

,source,target,solution,number of pathways,normalised values,higher yield than glucose
2,Vinyl chloride,Apigenin,9.379374,2,4.689687,1
4,Vinyl chloride,Luteolin,9.379374,2,4.689687,1
6,Vinyl chloride,Naringenin,9.379374,2,4.689687,1
10,Vinyl chloride,beta-Carotene,6.322964,8,3.161482,1
9,Vinyl chloride,beta-Carotene,6.126559,16,3.063280,1
0,Propene,Succinate,2.586895,1,0.862298,1
1,Propene,beta-Carotene,1.795530,2,0.598510,1
8,Vinyl chloride,beta-Carotene,1.795530,6,0.897765,1
3,Vinyl chloride,Eriodictyol,0.000000,1,0.000000,0
5,Vinyl chloride,Naringenin,0.000000,1,0.000000,0


In [16]:
# compare growth 09to glucose
grouped_growth_table['compared to glucose'] = 0.0

for i, row in grouped_growth_table.iterrows():
    grouped_growth_table.at[i, 'compared to glucose'] = row['normalised values'] / glucose_normalise
    
grouped_growth_table

,source,target,solution,number of pathways,normalised values,higher yield than glucose,compared to glucose
2,Vinyl chloride,Apigenin,9.379374,2,4.689687,1,343.441011
4,Vinyl chloride,Luteolin,9.379374,2,4.689687,1,343.441011
6,Vinyl chloride,Naringenin,9.379374,2,4.689687,1,343.441011
10,Vinyl chloride,beta-Carotene,6.322964,8,3.161482,1,231.525595
9,Vinyl chloride,beta-Carotene,6.126559,16,3.063280,1,224.333907
0,Propene,Succinate,2.586895,1,0.862298,1,63.148908
1,Propene,beta-Carotene,1.795530,2,0.598510,1,43.830831
8,Vinyl chloride,beta-Carotene,1.795530,6,0.897765,1,65.746247
3,Vinyl chloride,Eriodictyol,0.000000,1,0.000000,0,0.000000
5,Vinyl chloride,Naringenin,0.000000,1,0.000000,0,0.000000
